CNN = convolution relu mlp 순으로 적용

25쪽  
보통 cs분야에서 3d이상을 tensor라고 한다.  
CNN : 영상에 적합하긴 하지만 고정적인 사고를 가지지 말자..! RNN으로도 할 순 있다.  
Forwarding : 다음 layer로 이동하는거를 말하시는 것 같은데  
Why? MLP 논문에 써있음...  
영상이미지를 convolution 거치면서 변수를 만듦  

30쪽 :  
가로, 세로같은 크기로 kernel을 stride하자  

31쪽 :   
첫 번째 열도 같은 횟수로 kernel에서 사용될 수 있도록 하기위해서 padding함  

32쪽 :  
양수는 자기 값 그대로, 음수는 0로 만듦  
convolution이후 음각화된 부분을 없애고(평면이 됨) 양각화된 부분만 남음-> 사물을 인식하는데 더 도움된다.   

34쪽 :  
한 장의 사진에 여러 개의 필터를 적용해서 여러 개의 변형된 사진을 생성함(3D tensor).  
MLP에 적용시키기 위해서 flatten사용  

35쪽 :  
자동차 이미지에 흰 부분은 상관관계가 높은데, CNN을 적용해서 필요 없는 부분을 제거한다. [다중공선성 문제](https://nittaku.tistory.com/476)를 convolution이 해결한다고 하셨는데..?  --> 데이터의 일부만 사용해서 다중공선성 문제를 해결

> 다중공선성 해결방법  
변수선택 과정에서 상관계수가 높은 두 변수중에는 하나만을 선택 →   
종속변수 Y를 설명 하는데 중요한 역할을 하는 변수를 제외하지 않도록 주의  


출처: https://byzun0.tistory.com/90 [bevis study]


38쪽 : RNN  
날씨 예측의 경우, 전의 예측에 의존성이 존재...! (대부분 비오면 다음날 맑겠지..?)  

40쪽 :  
다음 들어갈 적절한 말이 label  

45 쪽 :   
현재의 상태에 영향을 줄법한 과거찾기  
앞 내용에서 쓸모 없는 것을 제거  

46 쪽 :  
![](./RNN_disadvantages.png)
RNN 단점 : 과거의 기억에 대한 가중치가 시간이 적을수록 적어진다.  
(정답인데도 기억 못함)  
=> 해결책 -> LSTM

48 쪽 :  
ci == mlp에 대한 cell이라 생각해도 좋다.  

50 쪽 :  
RNN을 적용하기 위해서 데이터의 전처리가 필요..!  

# 코드 설명
#### univariate_stacked_LSTM  
44쪽의 층이 여러 개  

#### tf_mlp_csv_mnist.ipynb
model을 hypothesis라고도 한다.  
Hypothesis = mlpModel(featureInputs = holderFeatures)  

[구글 검색 팁](https://strogay.tistory.com/entry/%EA%B5%AC%EA%B8%80%EB%A7%81-%EB%B0%8F-%EA%B5%AC%EA%B8%80-%EA%B2%80%EC%83%89-%ED%8C%81-21%EA%B0%80%EC%A7%80%EB%A1%9C-%EA%B5%AC%EA%B8%80%EA%B2%80%EC%83%89%EC%9D%98-%EB%8B%AC%EC%9D%B8%EB%90%98%EA%B8%B0-1)

In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
# keras one-hot encoding 방법

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# MLP구조
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 3s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2678 - acc: 0.9182 - val_loss: 0.0635 - val_acc: 0.9793
Epoch 2/12
60000/60000 [==============================] - 63s 1ms/step - loss: 0.0881 - acc: 0.9753 - val_loss: 0.0439 - val_acc: 0.9843
Epoch 3/12
60000/60000 [==============================] - 63s 1ms/step - loss: 0.0665 - acc: 0.9799 - val_loss: 0.0376 - val_acc: 0.9876
Epoch 4/12
60000/60000 [==============================] - 63s 1ms/step - loss: 0.0539 - acc: 0.9837 - val_loss: 0.0350 - val_acc: 0.9875
Epoch 5/12
60000/60000 [=====